In [1]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should return your GPU model


True
NVIDIA GeForce GTX 1650 with Max-Q Design


In [ ]:
#download the model and tokenizer

from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")

print("Model and tokenizer downloaded successfully!")


Some parameters are on the meta device because they were offloaded to the cpu.


Model and tokenizer downloaded successfully!


In [ ]:
#donwload the data

import pandas as pd
import glob

# Define the path to all CSV files in the "labelled data" folder
csv_files = glob.glob("data/reddit-mental-health-dataset/Original Reddit Data/Labelled Data/*.csv")

# Read all CSVs and combine them
df_list = [pd.read_csv(file) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)

# Show basic dataset info
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 823 entries, 0 to 822
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   score      800 non-null    float64
 1   selftext   800 non-null    object 
 2   subreddit  800 non-null    object 
 3   title      800 non-null    object 
 4   Label      800 non-null    object 
 5   CAT 1      200 non-null    object 
dtypes: float64(1), object(5)
memory usage: 38.7+ KB
None
   score                                           selftext subreddit  \
0    1.0  Tried to watch this documentary “anxious Ameri...   Anxiety   
1    1.0  i’m currently laying in bed wide awake, feelin...   Anxiety   
2    2.0  Second time trying weed. First time felt close...   Anxiety   
3    1.0  I am not posting this for me, but rather for m...   Anxiety   
4    1.0  21 year old male been dealing with anxiety eve...   Anxiety   

                                               title             Label CAT 1  


After downloading the data move the data to "data" folder and run the below code to get the data in the required format.

In [12]:
import pandas as pd
import glob

# Load all CSV files from the correct path
csv_files = glob.glob("data/reddit-mental-health-dataset/Original Reddit Data/Labelled Data/*.csv")

# Read and merge all CSVs
df_list = [pd.read_csv(file) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)

# Keep only relevant columns
df = df[['title', 'selftext', 'Label']]

# Drop missing values in 'selftext'
df = df.dropna(subset=['selftext'])

# Combine title and selftext into one column
df['text'] = df['title'].fillna('') + " " + df['selftext']

# Keep only the final processed text and label
df = df[['text', 'Label']]

# Show updated dataset info
print(df.info())
print(df.head())

# Save the cleaned dataset as a CSV file
df.to_csv("data/reddit-mental-health-dataset/cleaned_mental_health_data.csv", index=False)

print("✅ Cleaned dataset saved successfully!")


<class 'pandas.core.frame.DataFrame'>
Index: 800 entries, 0 to 822
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    800 non-null    object
 1   Label   800 non-null    object
dtypes: object(2)
memory usage: 18.8+ KB
None
                                                text             Label
0  Do people get over anxiety? Tried to watch thi...  Drug and Alcohol
1  does anyone else have this big fear of suddenl...  Drug and Alcohol
2  3 hour long panic attack after trying weed Sec...  Drug and Alcohol
3  Please leave in the comments ANYTHING that has...  Drug and Alcohol
4  Alcohol induced 21 year old male been dealing ...  Drug and Alcohol
✅ Cleaned dataset saved successfully!


In [13]:
from transformers import AutoTokenizer
import pandas as pd

# Load cleaned dataset
df = pd.read_csv("data/reddit-mental-health-dataset/cleaned_mental_health_data.csv")

# Load tokenizer for TinyLlama
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenization function
def tokenize_text(text):
    return tokenizer(text, padding="max_length", truncation=True, max_length=512)

# Apply tokenization
df["input_ids"] = df["text"].apply(lambda x: tokenize_text(str(x))["input_ids"])

# Keep only tokenized inputs and labels
df = df[["input_ids", "Label"]]

# Save tokenized data as Parquet for efficient processing
df.to_parquet("data/reddit-mental-health-dataset/tokenized_mental_health_data.parquet", engine="pyarrow")

print("✅ Tokenization complete! Data saved as Parquet.")


✅ Tokenization complete! Data saved as Parquet.


In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM
from datasets import load_dataset
import torch
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model
from transformers import BitsAndBytesConfig

# Enable 8-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,  # Use 8-bit quantization
    llm_int8_enable_fp32_cpu_offload=True  # Offload CPU computations to save VRAM
)

# Load model and force it onto the current CUDA device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map={'': torch.cuda.current_device()},  # Assign model to GPU
    quantization_config=quantization_config  # Apply quantization
)
  # Move model to GPU




# Apply LoRA for efficient fine-tuning
lora_config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["q_proj", "v_proj"],  
    lora_dropout=0.05, 
    bias="none"
)
model = get_peft_model(model, lora_config)

# Load tokenized dataset
from datasets import DatasetDict

# Load tokenized dataset
dataset = load_dataset("parquet", data_files={"data": "data/reddit-mental-health-dataset/tokenized_mental_health_data.parquet"})

# Convert dataset to dictionary format
dataset = dataset["data"].train_test_split(test_size=0.1)

# Ensure `labels` are the same as `input_ids`
dataset = DatasetDict({
    "train": dataset["train"].map(lambda x: {"labels": x["input_ids"]}),
    "eval": dataset["test"].map(lambda x: {"labels": x["input_ids"]})
})


# Define training arguments
training_args = TrainingArguments(
    output_dir="./fine_tuned_results",
    per_device_train_batch_size=2,  # Lower batch size for 4GB GPU
    per_device_eval_batch_size=2,
    num_train_epochs=3,  # Train for 3 epochs
    save_total_limit=2,
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=100,
    save_strategy="epoch"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["eval"]  # Add evaluation dataset
)


# Start training
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")
print("✅ Fine-tuning complete! Model saved.")


Map: 100%|██████████| 80/80 [00:00<00:00, 686.52 examples/s]


Epoch,Training Loss,Validation Loss
